In [ ]:
import sqlite3
import tkinter as tk
from tkinter import messagebox

# Step 1: Create/connect to SQLite database
conn = sqlite3.connect("soulsense_db")
cursor = conn.cursor()

# Step 2: Create table to store final score
cursor.execute("""
CREATE TABLE IF NOT EXISTS scores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    age INTEGER,
    total_score INTEGER
)
""")
conn.commit()

# Step 3: Define 20 Soul Sense questions
questions = [
    "You can recognize your emotions as they happen.",
    "You manage to stay calm even under pressure.",
    "You are aware of how your emotions affect others.",
    "You can motivate yourself to pursue long-term goals.",
    "You stay optimistic even after facing setbacks.",
    "You are able to express your feelings clearly.",
    "You handle conflict effectively and calmly.",
    "You find it easy to empathize with others' emotions.",
    "You adapt well to changing situations.",
    "You keep your promises and commitments.",
    "You actively listen to others when they speak.",
    "You accept constructive criticism without being defensive.",
    "You remain positive in emotionally difficult situations.",
    "You recognize stress in others and offer support.",
    "You maintain strong relationships over time.",
    "You can identify your emotional triggers.",
    "You take responsibility for your actions.",
    "You resolve misunderstandings diplomatically.",
    "You stay focused and productive when under pressure.",
    "You show appreciation and gratitude regularly."
]

# Step 4: GUI Application
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")
        self.username = ""
        self.age = None  # Optional age field
        self.current_question = 0
        self.responses = []

        self.create_username_screen()

    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=20)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack(pady=10)

        tk.Label(self.root, text="Enter Your Age (optional):", font=("Arial", 14)).pack(pady=10)
        self.age_entry = tk.Entry(self.root, font=("Arial", 14))
        self.age_entry.pack(pady=10)

        tk.Button(self.root, text="Start Test", command=self.start_test, font=("Arial", 12)).pack(pady=20)

    def validate_age_input(self, age_str):
        """Validate age input - returns (is_valid, age_value, error_message)"""
        if age_str == "":
            return (True, None, None)  # Age is optional
        try:
            age = int(age_str)
            if age < 1 or age > 120:
                return (False, None, "Please enter a valid age between 1 and 120.")
            return (True, age, None)
        except ValueError:
            return (False, None, "Age must be a numeric value. Please enter a valid number.")

    def start_test(self):
        self.username = self.name_entry.get().strip()
        age_str = self.age_entry.get().strip()
        
        if self.username == "":
            messagebox.showwarning("Input Error", "Please enter your name to start the test.")
            return
        
        # Validate age input
        is_valid, age_value, error_msg = self.validate_age_input(age_str)
        if not is_valid:
            messagebox.showwarning("Input Error", error_msg)
            return
        
        self.age = age_value
        self.show_question()

    def show_question(self):
        self.clear_screen()

        if self.current_question < len(questions):
            q_text = questions[self.current_question]
            tk.Label(self.root, text=f"Q{self.current_question + 1}: {q_text}", wraplength=400, font=("Arial", 14)).pack(pady=20)

            self.answer_var = tk.IntVar()

            for val, text in enumerate(["Never (1)", "Sometimes (2)", "Often (3)", "Always (4)"], start=1):
                tk.Radiobutton(self.root, text=text, variable=self.answer_var, value=val, font=("Arial", 12)).pack(anchor="w", padx=50)

            tk.Button(self.root, text="Next", command=self.save_answer, font=("Arial", 12)).pack(pady=20)

        else:
            self.finish_test()

    def save_answer(self):
        ans = self.answer_var.get()
        if ans == 0:
            messagebox.showwarning("Input Error", "Please select an answer before proceeding.")
        else:
            q_text = questions[self.current_question]
            self.responses.append((self.username, self.current_question + 1, q_text, ans))
            self.current_question += 1
            self.show_question()

    def finish_test(self):
        total_score = sum(r[3] for r in self.responses)

        # Store only final score in the database
        cursor.execute("INSERT INTO scores (username, age, total_score) VALUES (?, ?, ?)", (self.username, self.age, total_score))
        conn.commit()

        interpretation = ""

        if total_score >= 65:
            interpretation = "Excellent Emotional Intelligence!"
        elif total_score >= 50:
            interpretation = "Good Emotional Intelligence."
        elif total_score >= 35:
            interpretation = "Average Emotional Intelligence."
        else:
            interpretation = "You may want to work on your Emotional Intelligence."

        self.clear_screen()

        tk.Label(self.root, text=f"Thank you, {self.username}!", font=("Arial", 16)).pack(pady=10)
        tk.Label(self.root, text=f"Your total EQ score is: {total_score} / 80", font=("Arial", 14)).pack(pady=10)
        tk.Label(self.root, text=interpretation, font=("Arial", 14), fg="blue").pack(pady=10)

        # Show all results
        self.show_all_results()

        tk.Button(self.root, text="Exit", command=self.exit_test, font=("Arial", 12)).pack(pady=20)

    def show_all_results(self):
        # Display all results in the command line
        print("\n--- All EQ Test Results ---")
        print(f"{'Username':<20} {'Age':<10} {'Total Score'}")
        print("-" * 45)

        # Fetch all scores from the database
        cursor.execute("SELECT username, age, total_score FROM scores")
        rows = cursor.fetchall()

        for row in rows:
            age_display = str(row[1]) if row[1] is not None else "N/A"
            print(f"{row[0]:<20} {age_display:<10} {row[2]}")

    def exit_test(self):
        # Ensure database connection is closed first
        conn.close()
        self.root.quit()

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# Step 5: Main Loop
if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("500x300")
    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.exit_test) 
    root.mainloop()


--- All EQ Test Results ---
Username             Total Score
------------------------------
nupur                54
nishita              53
nupur                61
nishita              50
palak                63
nupur                58
chesta               56
himani               48
pihu                 58
